## Data Preparation

In [1]:
import numpy as np
import pandas as pd

In [3]:
cal_df = pd.read_csv('./Data/calendar.csv')

In [4]:
cal_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1393570 entries, 0 to 1393569
Data columns (total 4 columns):
 #   Column      Non-Null Count    Dtype 
---  ------      --------------    ----- 
 0   listing_id  1393570 non-null  int64 
 1   date        1393570 non-null  object
 2   available   1393570 non-null  object
 3   price       934542 non-null   object
dtypes: int64(1), object(3)
memory usage: 42.5+ MB


In [5]:
cal_df['listing_id'].nunique()

3818

In [9]:
listing_df = pd.read_csv('./Data/listings.csv')

In [10]:
listing_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3818 entries, 0 to 3817
Data columns (total 92 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   id                                3818 non-null   int64  
 1   listing_url                       3818 non-null   object 
 2   scrape_id                         3818 non-null   int64  
 3   last_scraped                      3818 non-null   object 
 4   name                              3818 non-null   object 
 5   summary                           3641 non-null   object 
 6   space                             3249 non-null   object 
 7   description                       3818 non-null   object 
 8   experiences_offered               3818 non-null   object 
 9   neighborhood_overview             2786 non-null   object 
 10  notes                             2212 non-null   object 
 11  transit                           2884 non-null   object 
 12  thumbn

In [11]:
listing_df.head(5)

,id,listing_url,scrape_id,last_scraped,name,summary,space,description,experiences_offered,neighborhood_overview,...,review_scores_value,requires_license,license,jurisdiction_names,instant_bookable,cancellation_policy,require_guest_profile_picture,require_guest_phone_verification,calculated_host_listings_count,reviews_per_month
0,241032,https://www.airbnb.com/rooms/241032,20160104002432,2016-01-04,Stylish Queen Anne Apartment,NaN,Make your self at home in this charming one-be...,Make your self at home in this charming one-be...,none,NaN,...,10.0,f,NaN,WASHINGTON,f,moderate,f,f,2,4.07
1,953595,https://www.airbnb.com/rooms/953595,20160104002432,2016-01-04,Bright & Airy Queen Anne Apartment,Chemically sensitive? We've removed the irrita...,"Beautiful, hypoallergenic apartment in an extr...",Chemically sensitive? We've removed the irrita...,none,"Queen Anne is a wonderful, truly functional vi...",...,10.0,f,NaN,WASHINGTON,f,strict,t,t,6,1.48
2,3308979,https://www.airbnb.com/rooms/3308979,20160104002432,2016-01-04,New Modern House-Amazing water view,New modern house built in 2013. Spectacular s...,"Our house is modern, light and fresh with a wa...",New modern house built in 2013. Spectacular s...,none,Upper Queen Anne is a charming neighborhood fu...,...,10.0,f,NaN,WASHINGTON,f,strict,f,f,2,1.15
3,7421966,https://www.airbnb.com/rooms/7421966,20160104002432,2016-01-04,Queen Anne Chateau,A charming apartment that sits atop Queen Anne...,NaN,A charming apartment that sits atop Queen Anne...,none,NaN,...,NaN,f,NaN,WASHINGTON,f,flexible,f,f,1,NaN
4,278830,https://www.airbnb.com/rooms/278830,20160104002432,2016-01-04,Charming craftsman 3 bdm house,Cozy family craftman house in beautiful neighb...,Cozy family craftman house in beautiful neighb...,Cozy family craftman house in beautiful neighb...,none,We are in the beautiful neighborhood of Queen ...,...,9.0,f,NaN,WASHINGTON,f,strict,f,f,1,0.89


In [13]:
review_df = pd.read_csv('./Data/reviews.csv')

In [14]:
review_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 84849 entries, 0 to 84848
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   listing_id     84849 non-null  int64 
 1   id             84849 non-null  int64 
 2   date           84849 non-null  object
 3   reviewer_id    84849 non-null  int64 
 4   reviewer_name  84849 non-null  object
 5   comments       84831 non-null  object
dtypes: int64(3), object(3)
memory usage: 3.9+ MB


## Question 1: If a couple wants to stay in a entire apartment or house, which neighborhood group would most people pick? (Assuming the number of reviews represents for the number of bookings)

In [15]:
list_df = listing_df[(listing_df['accommodates'] ==2)&(listing_df['room_type'] =='Entire home/apt')] 
list_df.groupby(['neighbourhood_group_cleansed'])['number_of_reviews'].sum().sort_values(ascending = False)

# I picked neighborhood group because it's easy for tourists to understand seattle neighborhood in a high level

# Top3 neighborhood groups would be Capitol Hill, Downtown and Ballard

neighbourhood_group_cleansed
Capitol Hill           3345
Other neighborhoods    2958
Downtown               2356
Ballard                1850
Queen Anne             1746
Central Area           1729
Rainier Valley          827
West Seattle            644
Delridge                589
Cascade                 549
Beacon Hill             456
Northgate               334
Lake City               219
University District     217
Seward Park             130
Magnolia                 48
Interbay                  3
Name: number_of_reviews, dtype: int64

In [16]:
# Question 1.2: How's the average rating and options of top 3 popular neighborhoods look like?

In [17]:
a = ["Capitol Hill","Downtown","Ballard"]

list_df1 = list_df[list_df['neighbourhood_group_cleansed'].isin(a)]
list_df1.groupby(['neighbourhood_group_cleansed'])['review_scores_rating'].mean().sort_values(ascending = False)

neighbourhood_group_cleansed
Ballard         95.536585
Capitol Hill    93.657143
Downtown        92.747368
Name: review_scores_rating, dtype: float64

In [18]:
list_df1.groupby(['neighbourhood_group_cleansed'])['review_scores_rating'].count().sort_values(ascending = False)

neighbourhood_group_cleansed
Capitol Hill    140
Downtown         95
Ballard          41
Name: review_scores_rating, dtype: int64

In [19]:
# Ballard neighborhood has highest ratings but relatively less options while capitol hill and downtown offers more options.

## Question 2: What should the couple expect about the availiability and price range of the airbnbs in the popular neighrborhoods if they wants to stay for a long weekend?

In [20]:
pd.set_option('mode.chained_assignment', None)

In [21]:
#filter the available options
list_df2 = list_df1[(list_df1['minimum_nights']<4)&(list_df1['host_acceptance_rate'] == "100%")]

# change the data type of the price column
list_df2['price'] = list_df2['price'].replace('[\$,]', '', regex=True).astype(float)
list_df2['cleaning_fee'] = list_df2['cleaning_fee'].replace('[\$,]', '', regex=True).astype(float)

In [22]:
# For some columns, the cleaning fee value is missing, so calculate the average ratio of cleaning fee/price here

list_df2['cleaning_fee_ratio'] = list_df2['cleaning_fee']/list_df2['price']
c = list_df2['cleaning_fee_ratio'] .mean()

# the raio is about 42%, so we are using this to impute the missing value

list_df2.loc[list_df2['cleaning_fee'].isna() == True,'cleaning_fee'] = list_df2['price']*c

In [23]:
# total price = price * 2 + cleaning fee
list_df2['weekend_total'] = list_df2['price']*3 + list_df2['cleaning_fee']

In [24]:
# summary for the price range
# Here I picked neighbourhood itself instead of neighborhood group because we want to know more about detailed level

weekend_min = list_df2.groupby(['neighbourhood_cleansed'])['weekend_total'].min().rename('Min_price')
weekend_avg = list_df2.groupby(['neighbourhood_cleansed'])['weekend_total'].mean().rename('Mean_price')
weekend_max = list_df2.groupby(['neighbourhood_cleansed'])['weekend_total'].max().rename('Max_price')
Avaliable_options = list_df2.groupby(['neighbourhood_cleansed'])['weekend_total'].count().rename('Avaliable_options')

In [25]:
list_df3 = pd.concat([weekend_min,weekend_avg,weekend_max,Avaliable_options],axis=1)

In [26]:
list_df3.sort_values('Avaliable_options',ascending=False)

,Min_price,Mean_price,Max_price,Avaliable_options
neighbourhood_cleansed,,,,
Broadway,192.000000,351.029704,857.742350,92
Belltown,235.000000,427.710808,630.000000,41
First Hill,245.000000,416.099414,625.000000,20
Stevens,187.000000,372.544421,686.193880,17
Adams,247.000000,372.834942,575.000000,15
Central Business District,202.427195,441.071801,1029.290820,10
Loyal Heights,223.013011,298.848232,414.000000,9
West Woodland,257.322705,348.122297,497.000000,7
Pioneer Square,230.000000,551.206524,850.000000,7


In [27]:
list_df3 = list_df3.reset_index()

## Question 3: Would booking a more expensive place really worth it for this couple?

In [28]:
# Let's say we want to see at least 10 available options in a certain neighborhood so we can compare different options
d = list_df3[list_df3['Avaliable_options']>10]['neighbourhood_cleansed']
list_df4 = list_df2[list_df2['neighbourhood_cleansed'].isin(d)]

In [29]:
# Get the price dist

list_df4['weekend_total'].quantile([0.25,0.5,0.75])

0.25    291.632399
0.50    360.251787
0.75    460.000000
Name: weekend_total, dtype: float64

In [30]:
bins = [0,292,360,460,1000]
list_df4['price_group'] = pd.cut(list_df4['weekend_total'], bins)

In [31]:
avg_rating = list_df4.groupby(['price_group'])['review_scores_rating'].mean().rename('avg_rating')
avg_accuracy = list_df4.groupby(['price_group'])['review_scores_accuracy'].mean().rename('avg_accuracy')
avg_cleanliness = list_df4.groupby(['price_group'])['review_scores_cleanliness'].mean().rename('avg_cleanliness')
avg_checkin = list_df4.groupby(['price_group'])['review_scores_checkin'].mean().rename('avg_checkin')
avg_communication = list_df4.groupby(['price_group'])['review_scores_communication'].mean().rename('avg_communication')
avg_location = list_df4.groupby(['price_group'])['review_scores_location'].mean().rename('avg_location')
avg_value = list_df4.groupby(['price_group'])['review_scores_value'].mean().rename('avg_value')
option_counts = list_df4.groupby(['price_group'])['review_scores_rating'].count().rename('option_counts')

In [32]:
list_df5 = pd.concat([avg_rating,avg_accuracy,avg_cleanliness,avg_checkin,avg_communication,avg_location,avg_value,option_counts],axis=1)

In [33]:
list_df5

,avg_rating,avg_accuracy,avg_cleanliness,avg_checkin,avg_communication,avg_location,avg_value,option_counts
price_group,,,,,,,,
"(0, 292]",91.400000,9.100000,9.325000,9.675000,9.750000,9.850000,9.325000,40
"(292, 360]",95.459459,9.783784,9.567568,9.864865,9.891892,9.891892,9.567568,37
"(360, 460]",91.931818,9.454545,9.409091,9.636364,9.568182,9.772727,9.204545,44
"(460, 1000]",94.909091,9.636364,9.681818,9.886364,9.954545,9.909091,9.295455,44


In [34]:
# paying more than $292 makes a difference on the accuracy and cleanliness, but wouldn't necessarily impact the checkin, communication, and location
# paying more than $360 might not be worth it since the previous customer thought it wasn't worth the value
# overall customers paying between 292 and 360 are happiest about the airbnb.